In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import pickle


tox = pd.read_excel('envirotox_20240201154635.xlsx')
inp_tox = tox[(tox['Test statistic'] == 'EC50') & (tox['Duration (days)'] == 4)]
df_kosh = pd.read_csv('nist_ir_spectra_list_with_bonds.csv')

with open('saved_dictionary.pkl', 'rb') as handle:
    smiles_dict = pickle.load(handle)
smiles_list = []
for i,row in inp_tox.iterrows():
    smiles_list.append(smiles_dict[row['Chemical name'].split(';')[0]])
inp_tox.insert(0, 'smiles', smiles_list)
column_list = [i for i in df_kosh.columns if '─' in i]

corr_matrix = df_kosh[column_list].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.75)] 
df_kosh.drop(to_drop, axis=1, inplace=True)

column_list = [i for i in df_kosh.columns if '─' in i]
drop_list = []
for i in column_list:
    if df_kosh[i].sum() == 0:
        drop_list.append(i)

df_kosh.drop(drop_list, axis=1, inplace=True)
dataset = inp_tox.merge(df_kosh, right_on='SMILES', left_on='smiles')
dataset.to_csv('EC50_flags.csv', index=False)